# Complexity analysis - Practical session 2: NP Completness and the backpack/knapsack problem

In this TP we propose to examine different ways of coding and solving problems of the NP class. We are interested in the problem of the backpack.
We suppose that we have $l$ items each having a utility (or gain) $u_i$. Each of these items has a weight $m_i$. We try to maximize the gain by packing
as many items as possible in a maximum capacity bag $M$. We distinguish two interesting cases :
 - the items are only available in one copy, i.e. we are trying to determine the quantity $x_i \in \{0,1\}$ associated with each item, 
 - one can take several times the same item, i.e. $x_i \in \mathbb{N}^+$. The problem is formalized as follows:
\begin{aligned}
U & =  \text{max}_{x_i} \sum_i x_i u_i \\\
    & \text{s.c.} \sum_i x_i m_i \leq M
\end{aligned}

We will examine different methods to solve this problem allowing you to feel its complexity. You generate for each test that you will make a utility vector and a vector 
of weights that will be randomly drawn integers in $[1.10]$. You will set $M$ according to the number of possible items, for example if you have $n$ items (which will be a 
parameter of your test procedure), you will be able to choose $M=7n$. You will write a function `solve_bag` for each variant that will take the utility, weight and $M$ vectors as parameters and 
will return the maximum value (total gain) reached, as well as the time related to the calculation. 

Write the code below that will generate the possible data for this problem.


In [30]:
import numpy as np

n = 5
utility = np.random.randint(1, 10, n)
weight = np.random.randint(1, 10, n)
M = 7 * n
print('utility : ' + str(utility))
print('weight : ' + str(weight))
print('M : ' + str(M))

utility : [6 6 7 3 4]
weight : [5 9 6 4 4]
M : 35


## Brute force approach

We don't bother with complex considerations here: write a method that calculates all possible combinations ($2^n$!), evaluates them, and returns the optimal gain. 


In [32]:
def solve_bag(M, utility, weight, n):
    if (n == 0) or (M == 0):
        return 0
    if weight[n - 1] > M:
        return solve_bag(M, utility, weight, n - 1)
    else:
        return max(utility[n - 1] + solve_bag(M - weight[n - 1], utility, weight, n - 1), solve_bag(M, utility, weight, n- 1))

print('solve_bag : ' + str(solve_bag(M, utility, weight, n)))

solve_bag : 26


Same thing but this time you can choose the same item several times. For each item we can determine the maximum limit of the number of times we can choose this item as the integer part of 
$M/m_i$. Be careful, calculation times can become very long for high $M$ values.

In [37]:
def solve_bag_unlimited_items(M, utility, weight, n, maxUse):
    if (n <= 0) or (maxUse > n) or (maxUse <= 0):
        return 0
    result = 0
    for i in range(n):
        if M - weight[i] >= 0:
            result = max(result, utility[i] + solve_bag_unlimited_items(M - weight[i], utility, weight, n, maxUse + 1))
    return result

print('solve_bag_maxUse() : ' + str(solve_bag_unlimited_items(M, utility, weight, n, 0)))

solve_bag_maxUse() : 21


## Greedy approach

For each object the gain/mass ratio ($u_i/m_i$) is calculated. The objects are sorted in descending order, then the bag is filled in this order until no more items can be added.
Compare the quality of the solution obtained with that of the previous exact solver. In particular, find cases where the gluttonous strategy does not give the optimal solution of the 
problem. Here again you will code two versions of the function (only one item available and unlimited number of items available). 


In [51]:
def solve_bag_greedy(M, utility, weight, n):
    sorted_utility = sorted(utility, reverse = True)
    sorted_weight = sorted(weight, reverse = True)
    i = 0
    result = 0
    while (M > 0) and (i < n):
        if (M - sorted_weight[i]) >= 0:
            result += sorted_utility[i]
            M -= sorted_weight[i]
        i += 1
    return result

def solve_bag_greedy_unlimited_items(M, utility, weight, n, maxUse):
    sorted_utility = sorted(utility, reverse = True)
    sorted_weight = sorted(weight, reverse = True)
    result = 0
    i = 0
    while (M > 0) and (i < n) and (maxUse < n):
        if (M - sorted_weight[i]) >= 0:
            result += sorted_utility[i]
            M -= sorted_weight[i]
            maxUse += 1
        i += 1
    return result

print('solve_bag_greedy() : ' + str(solve_bag_greedy(M, utility, weight, n)))
print('solve_bag_greedy_unlimited_items() : ' + str(solve_bag_greedy_unlimited_items(M, utility, weight, n, 0)))

solve_bag_greedy() : 26
solve_bag_greedy_unlimited_items() : 26


## Dynamic programming

We limit ourselves here to the case where only one item of each object is available.

The idea of dynamic programming is to solve incrementally simpler versions of the problem, and to store intermediate results necessary to 
add new variables. A time-space compromise is then made. In the case of the backpack problem, the problem is said to be with *optimal substructure*, that is to say that we 
can find the optimal value of the problem at variable $i$ from the optimal value at variable $i-1$. 
The following quantity $P(k,m)$, describing the state of the system for k variables, is defined by recurrence :
$$
\begin{eqnarray}
P(k,m) & = & \text{max}_{x_i} \sum^k_i x_i u_i \\
    &    & \text{s.c.} \sum_i x_i m_i \leq m
\end{eqnarray}
$$
then the optimal solution is either :
 - the optimal solution $P(k-1,m)$ where one chooses not to add the item, i.e. $x_k=0$.
- the optimal solution $P(k-1,m-m_k)+u_k$ where we choose to add the item, i.e. $x_k=1$.

You just have to build a table of the different possibilities $P(k,m)$. Once this table is built, you just have to start from the element 
$P(k,M)$ and go back to the $P(0,.)$ element to find out whether you choose the item or not and thus construct the solution.

We note then that the complexity of the algorithm is in time and space $o(nM)$. Although polynomial, we have not shown that $P=NP$: 
since the coding of $M$ is done on $log(M)$ bits, we remain within the exponential complexity of the size of the input.
  

Code and test this algorithm.

Optionnaly, you can get some help from the Wikipedia page on [0-1 knapasack problem](https://en.wikipedia.org/wiki/Knapsack_problem).

In [48]:
def solve_bag_dynamic(M, utility, weight, n):
    result = [[0 for k in range(M + 1)] for k in range(n + 1)]
    for i in range(n + 1):
        for j in range(M + 1):
            if (i == 0) or (j == 0):
                result[i][j] = 0
            elif weight[i - 1] <= j:
                result[i][j] = max(result[i - 1][j - 1], utility[i - 1] + result[i - 1][j - weight[i - 1]])
            else:
                result[i][j] = result[i - 1][j]
    return result[n][M]

solve_bag_dynamic() : 26


## Summary and final conclusions
For each method, vary the number of items $n$, and measure an average time taken on $10$ resolution of the problem. Draw the corresponding average execution time curves
for the 3 methods in both cases (one or more times the same item, where dynamic programming will be excluded). 